In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import tensorflow as tf
import peakutils
from sklearn.linear_model import LogisticRegression

#from mpl_toolkits.mplot3d import Axes3D

sensorData = []
soundData  = []
labels = []

punch0 = []
punch1 = []
punch2 = []
punch3 = []
punch4 = []


 
with open ('sensors1.csv', 'r') as readFile:
    newFileReader = csv.reader(readFile)
    for row in readFile:
        line = []
        for x in row.split(','):
            line1 = []
            for y in x.split(' '):
                try:
                    line1.append(float(y))
                except Exception:
                    pass
            if len(line1) == 6:
                line.append(line1)
        if len(line) == 100:
            sensorData.append(line)
with open ('sounds1.csv', 'r') as readFile:
    newFileReader = csv.reader(readFile)
    for row in readFile:
        empty = []
        for i in row.split(','):
            empty.append(float(i))
        soundData.append(empty)
with open ('labels1.csv', 'r') as readFile:
    newFileReader = csv.reader(readFile)
    for row in readFile:
        labels.append(int(row))

for x in range(len(sensorData)):
    if x % 5 == 0:
        punch0.append(sensorData[x])
    if x % 5 == 1:
        punch1.append(sensorData[x])
    if x % 5 == 2:
        punch2.append(sensorData[x])
    if x % 5 == 3:
        punch3.append(sensorData[x])
    if x % 5 == 4:
        punch4.append(sensorData[x])
soundData  = np.array(soundData)
sensorData = np.array(sensorData)
labels     = np.array(labels)
punch0 = np.array(punch0)
punch1 = np.array(punch1)
punch2 = np.array(punch2)
punch3 = np.array(punch3)
punch4 = np.array(punch4)




In [2]:
from scipy.fftpack import fft, rfft
sensorData = np.array(sensorData)

def zeroCrossings(x):
    zero_crossings = np.where(np.diff(np.sign(x)))[0]
    return(len(zero_crossings))
    

def featureExtraction(x, sound):
    features = []
    for u in range(6):
        features.append(np.mean(x[:100, u]))
        features.append(np.max(x[:100, u]))
        features.append(np.min(x[:100, u]))
        features.append(np.std(x[:100, u]))
        features.append(zeroCrossings(x[:100, u]))
        features.append(np.var(x[:100, u]))
    features.append(np.std(x[:100, :3]))
    features.append(np.std(x[:100, 3:]))
    a, b, c, d, e, f, g, h, i = topPeaks(sound)
    features.append(a)
    features.append(b)
    features.append(c)
    features.append(d)
    features.append(e)
    features.append(f)
    features.append(g)
    features.append(h)
    features.append(i)
    return features

def topPeaks(sound):
    fourier = abs(rfft(sound))
    phase = (np.angle(rfft(sound)))
    indices = peakutils.indexes(fourier, thres=0.02/max(fourier), min_dist=0.1)
    amplitudes = (fourier[indices])
    amplitudes = (np.sort(amplitudes))[len(amplitudes)-3:len(amplitudes)]
    a    = fourier.tolist().index(amplitudes[0])
    b    = fourier.tolist().index(amplitudes[1])
    c    = fourier.tolist().index(amplitudes[2])
    d, e, f = amplitudes
    g    = phase[a]
    h    = phase[b]
    i    = phase[c]
    return a, b, c, d, e, f, g, h, i

topPeaks(soundData[250,0:])
# for x in range(len(sensorData)):
#     print(x)
#     print(featureExtraction(sensorData[x], soundData[x, 0:]))  
features = []

for x in range(len(sensorData)):
     features.append(featureExtraction(sensorData[x], soundData[x, 0:]))

features = np.array(features)

training = features[100:]
test     = features[:100]
training_labels = labels[100:]
test_labels = labels[:100]



def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation]
    shuffled_labels  = labels[permutation]
    return shuffled_dataset, shuffled_labels


train_dataset, train_labels = randomize(training, training_labels)
test_dataset, test_labels = randomize(test, test_labels)




In [7]:
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.contrib import learn

feature_columns = learn.infer_real_valued_columns_from_input(train_dataset)
classifier = learn.LinearClassifier(n_classes = 5, feature_columns=feature_columns)
#classifier = learn.DNNClassifier(feature_columns=feature_columns, hidden_units=[10, 20, 10], n_classes=5)
classifier.fit(train_dataset, train_labels, steps=100000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb586f3b5c0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpb0fskjgo'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpb0fskjgo/model.ckpt.
INFO:tensorflow:loss = 1.609438, st

INFO:tensorflow:loss = 802935.44, step = 7100 (0.133 sec)
INFO:tensorflow:global_step/sec: 656.576
INFO:tensorflow:loss = 305608.9, step = 7200 (0.153 sec)
INFO:tensorflow:global_step/sec: 662.512
INFO:tensorflow:loss = 183522.11, step = 7300 (0.151 sec)
INFO:tensorflow:global_step/sec: 658.771
INFO:tensorflow:loss = 1030139.75, step = 7400 (0.151 sec)
INFO:tensorflow:global_step/sec: 660.327
INFO:tensorflow:loss = 705539.7, step = 7500 (0.151 sec)
INFO:tensorflow:global_step/sec: 569.254
INFO:tensorflow:loss = 211700.2, step = 7600 (0.176 sec)
INFO:tensorflow:global_step/sec: 722.239
INFO:tensorflow:loss = 183062.61, step = 7700 (0.138 sec)
INFO:tensorflow:global_step/sec: 743.307
INFO:tensorflow:loss = 184473.1, step = 7800 (0.134 sec)
INFO:tensorflow:global_step/sec: 818.644
INFO:tensorflow:loss = 195947.92, step = 7900 (0.122 sec)
INFO:tensorflow:global_step/sec: 905.385
INFO:tensorflow:loss = 194317.88, step = 8000 (0.110 sec)
INFO:tensorflow:global_step/sec: 904.095
INFO:tensorfl

INFO:tensorflow:global_step/sec: 866.035
INFO:tensorflow:loss = 84743.95, step = 15400 (0.116 sec)
INFO:tensorflow:global_step/sec: 937.463
INFO:tensorflow:loss = 96483.2, step = 15500 (0.107 sec)
INFO:tensorflow:global_step/sec: 903.514
INFO:tensorflow:loss = 471249.12, step = 15600 (0.111 sec)
INFO:tensorflow:global_step/sec: 901.623
INFO:tensorflow:loss = 121396.09, step = 15700 (0.111 sec)
INFO:tensorflow:global_step/sec: 880.302
INFO:tensorflow:loss = 88376.4, step = 15800 (0.113 sec)
INFO:tensorflow:global_step/sec: 919.836
INFO:tensorflow:loss = 73636.625, step = 15900 (0.109 sec)
INFO:tensorflow:global_step/sec: 891.895
INFO:tensorflow:loss = 112373.63, step = 16000 (0.113 sec)
INFO:tensorflow:global_step/sec: 796.456
INFO:tensorflow:loss = 87297.94, step = 16100 (0.125 sec)
INFO:tensorflow:global_step/sec: 780.183
INFO:tensorflow:loss = 53301.52, step = 16200 (0.128 sec)
INFO:tensorflow:global_step/sec: 774.991
INFO:tensorflow:loss = 64366.176, step = 16300 (0.129 sec)
INFO:te

INFO:tensorflow:loss = 189502.8, step = 23600 (0.137 sec)
INFO:tensorflow:global_step/sec: 773.661
INFO:tensorflow:loss = 369620.2, step = 23700 (0.128 sec)
INFO:tensorflow:global_step/sec: 936.369
INFO:tensorflow:loss = 35840.145, step = 23800 (0.107 sec)
INFO:tensorflow:global_step/sec: 801.198
INFO:tensorflow:loss = 44684.8, step = 23900 (0.125 sec)
INFO:tensorflow:global_step/sec: 591.297
INFO:tensorflow:loss = 114571.94, step = 24000 (0.169 sec)
INFO:tensorflow:global_step/sec: 749.534
INFO:tensorflow:loss = 80325.94, step = 24100 (0.133 sec)
INFO:tensorflow:global_step/sec: 781.08
INFO:tensorflow:loss = 37788.45, step = 24200 (0.128 sec)
INFO:tensorflow:global_step/sec: 723.952
INFO:tensorflow:loss = 73284.29, step = 24300 (0.138 sec)
INFO:tensorflow:global_step/sec: 907.393
INFO:tensorflow:loss = 88611.28, step = 24400 (0.110 sec)
INFO:tensorflow:global_step/sec: 919.249
INFO:tensorflow:loss = 46085.793, step = 24500 (0.109 sec)
INFO:tensorflow:global_step/sec: 855.082
INFO:tens

INFO:tensorflow:global_step/sec: 785.293
INFO:tensorflow:loss = 246764.27, step = 31900 (0.127 sec)
INFO:tensorflow:global_step/sec: 748.57
INFO:tensorflow:loss = 119649.73, step = 32000 (0.134 sec)
INFO:tensorflow:global_step/sec: 763.075
INFO:tensorflow:loss = 35953.953, step = 32100 (0.131 sec)
INFO:tensorflow:global_step/sec: 781.179
INFO:tensorflow:loss = 38589.137, step = 32200 (0.128 sec)
INFO:tensorflow:global_step/sec: 795.002
INFO:tensorflow:loss = 118920.96, step = 32300 (0.126 sec)
INFO:tensorflow:global_step/sec: 758.551
INFO:tensorflow:loss = 62430.96, step = 32400 (0.132 sec)
INFO:tensorflow:global_step/sec: 778.707
INFO:tensorflow:loss = 79915.805, step = 32500 (0.128 sec)
INFO:tensorflow:global_step/sec: 761.349
INFO:tensorflow:loss = 44069.2, step = 32600 (0.131 sec)
INFO:tensorflow:global_step/sec: 769.528
INFO:tensorflow:loss = 40051.6, step = 32700 (0.130 sec)
INFO:tensorflow:global_step/sec: 744.101
INFO:tensorflow:loss = 91447.805, step = 32800 (0.135 sec)
INFO:t

...

INFO:tensorflow:loss = 34430.03, step = 73100 (0.138 sec)
INFO:tensorflow:global_step/sec: 719.561
INFO:tensorflow:loss = 19694.768, step = 73200 (0.139 sec)
INFO:tensorflow:global_step/sec: 744.436
INFO:tensorflow:loss = 18399.12, step = 73300 (0.134 sec)
INFO:tensorflow:global_step/sec: 750.431
INFO:tensorflow:loss = 23141.871, step = 73400 (0.133 sec)
INFO:tensorflow:global_step/sec: 744.368
INFO:tensorflow:loss = 27405.664, step = 73500 (0.134 sec)
INFO:tensorflow:global_step/sec: 729.4
INFO:tensorflow:loss = 16456.031, step = 73600 (0.137 sec)
INFO:tensorflow:global_step/sec: 722.731
INFO:tensorflow:loss = 19129.408, step = 73700 (0.138 sec)
INFO:tensorflow:global_step/sec: 787.199
INFO:tensorflow:loss = 62815.44, step = 73800 (0.127 sec)
INFO:tensorflow:global_step/sec: 805.497
INFO:tensorflow:loss = 32659.104, step = 73900 (0.124 sec)
INFO:tensorflow:global_step/sec: 805.828
INFO:tensorflow:loss = 30616.24, step = 74000 (0.124 sec)
INFO:tensorflow:global_step/sec: 809.635
INFO:t

INFO:tensorflow:global_step/sec: 748.978
INFO:tensorflow:loss = 17590.465, step = 81400 (0.133 sec)
INFO:tensorflow:global_step/sec: 798.172
INFO:tensorflow:loss = 25361.408, step = 81500 (0.125 sec)
INFO:tensorflow:global_step/sec: 792.534
INFO:tensorflow:loss = 27001.615, step = 81600 (0.126 sec)
INFO:tensorflow:global_step/sec: 848.205
INFO:tensorflow:loss = 17145.855, step = 81700 (0.118 sec)
INFO:tensorflow:global_step/sec: 785.755
INFO:tensorflow:loss = 13074.832, step = 81800 (0.128 sec)
INFO:tensorflow:global_step/sec: 846.638
INFO:tensorflow:loss = 20760.352, step = 81900 (0.118 sec)
INFO:tensorflow:global_step/sec: 818.398
INFO:tensorflow:loss = 34916.13, step = 82000 (0.122 sec)
INFO:tensorflow:global_step/sec: 840.119
INFO:tensorflow:loss = 15886.784, step = 82100 (0.119 sec)
INFO:tensorflow:global_step/sec: 769.707
INFO:tensorflow:loss = 16842.96, step = 82200 (0.130 sec)
INFO:tensorflow:global_step/sec: 751.311
INFO:tensorflow:loss = 36104.19, step = 82300 (0.133 sec)
INF

INFO:tensorflow:loss = 27231.344, step = 89600 (0.122 sec)
INFO:tensorflow:global_step/sec: 805.412
INFO:tensorflow:loss = 21193.744, step = 89700 (0.125 sec)
INFO:tensorflow:global_step/sec: 740.079
INFO:tensorflow:loss = 20233.473, step = 89800 (0.134 sec)
INFO:tensorflow:global_step/sec: 752.643
INFO:tensorflow:loss = 19508.432, step = 89900 (0.133 sec)
INFO:tensorflow:global_step/sec: 797.832
INFO:tensorflow:loss = 12526.944, step = 90000 (0.125 sec)
INFO:tensorflow:global_step/sec: 812.313
INFO:tensorflow:loss = 23560.32, step = 90100 (0.123 sec)
INFO:tensorflow:global_step/sec: 805.077
INFO:tensorflow:loss = 19794.816, step = 90200 (0.124 sec)
INFO:tensorflow:global_step/sec: 661.854
INFO:tensorflow:loss = 45892.03, step = 90300 (0.152 sec)
INFO:tensorflow:global_step/sec: 772.477
INFO:tensorflow:loss = 18341.328, step = 90400 (0.129 sec)
INFO:tensorflow:global_step/sec: 796.161
INFO:tensorflow:loss = 13794.8, step = 90500 (0.126 sec)
INFO:tensorflow:global_step/sec: 834.807
INFO

INFO:tensorflow:global_step/sec: 833.704
INFO:tensorflow:loss = 31530.96, step = 97900 (0.120 sec)
INFO:tensorflow:global_step/sec: 834.201
INFO:tensorflow:loss = 16755.92, step = 98000 (0.120 sec)
INFO:tensorflow:global_step/sec: 821.791
INFO:tensorflow:loss = 21165.68, step = 98100 (0.122 sec)
INFO:tensorflow:global_step/sec: 805.706
INFO:tensorflow:loss = 22704.672, step = 98200 (0.124 sec)
INFO:tensorflow:global_step/sec: 809.433
INFO:tensorflow:loss = 17124.385, step = 98300 (0.124 sec)
INFO:tensorflow:global_step/sec: 835.186
INFO:tensorflow:loss = 16919.615, step = 98400 (0.118 sec)
INFO:tensorflow:global_step/sec: 800.681
INFO:tensorflow:loss = 14109.568, step = 98500 (0.125 sec)
INFO:tensorflow:global_step/sec: 798.769
INFO:tensorflow:loss = 18387.184, step = 98600 (0.125 sec)
INFO:tensorflow:global_step/sec: 797.522
INFO:tensorflow:loss = 16591.84, step = 98700 (0.125 sec)
INFO:tensorflow:global_step/sec: 842.91
INFO:tensorflow:loss = 111896.46, step = 98800 (0.119 sec)
INFO:

LinearClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x7fb586f3b470>, 'feature_columns': [_RealValuedColumn(column_name='', dimension=47, default_value=None, dtype=tf.float64, normalizer=None)], 'optimizer': None, 'gradient_clip_norm': None, 'joint_weights': False})

In [6]:
classifier.evaluate(test_dataset, test_labels)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Starting evaluation at 2018-07-18-09:24:23
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpzdvlrpz3/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-18-09:24:24
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.44, global_step 

{'loss': 3186531.8, 'accuracy': 0.44, 'global_step': 1000}